*Reference to: https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix*

In [3]:
import os
import time
from options.train_options import TrainOptions
from options.test_options import TestOptions
from data import create_dataset
from models import create_model
from util.visualizer import Visualizer, save_images
from util import html

#### Далее ниже в качестве базового GAN'а будет использоваться LSGAN.

## pix2pix

In [4]:
opt_pix2pix = TrainOptions().parse()

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./facades/                    
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              epoch_count: 1  

In [5]:
opt_pix2pix.model = "pix2pix"
opt_pix2pix.dataset_mode = "aligned"
opt_pix2pix.name = "pix2pix: LSGAN"
opt_pix2pix.lambda_L1 = 100.0

In [6]:
dataset = create_dataset(opt_pix2pix)
dataset_size = len(dataset)    # get the number of images in the dataset.
print('The number of training images = %d' % dataset_size)

dataset [AlignedDataset] was created
The number of training images = 400


In [7]:
model = create_model(opt_pix2pix)      # create a model given opt.model and other options
model.setup(opt_pix2pix)               # regular setup: load and print networks; create schedulers
visualizer = Visualizer(opt_pix2pix)   # create a visualizer that display/save images and plots
total_iters = 0 

Setting up a new session...


initialize network with normal
initialize network with normal
model [Pix2PixModel] was created
---------- Networks initialized -------------
[Network G] Total number of parameters : 11.378 M
[Network D] Total number of parameters : 2.768 M
-----------------------------------------------
create web directory ./checkpoints/pix2pix: LSGAN/web...


In [8]:
for epoch in range(opt_pix2pix.epoch_count, opt_pix2pix.n_epochs + opt_pix2pix.n_epochs_decay + 1):    # outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
    epoch_start_time = time.time()  # timer for entire epoch
    iter_data_time = time.time()    # timer for data loading per iteration
    epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch
    visualizer.reset()              # reset the visualizer: make sure it saves the results to HTML at least once every epoch
    model.update_learning_rate()    # update learning rates in the beginning of every epoch.
    for i, data in enumerate(dataset):  # inner loop within one epoch
        iter_start_time = time.time()  # timer for computation per iteration
        if total_iters % opt_pix2pix.print_freq == 0:
            t_data = iter_start_time - iter_data_time

        total_iters += opt_pix2pix.batch_size
        epoch_iter += opt_pix2pix.batch_size
        model.set_input(data)         # unpack data from dataset and apply preprocessing
        model.optimize_parameters()   # calculate loss functions, get gradients, update network weights

        if total_iters % opt.display_freq == 0:   # display images on visdom and save images to a HTML file
            save_result = total_iters % opt_pix2pix.update_html_freq == 0
            model.compute_visuals()
            visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

        if total_iters % opt.print_freq == 0:    # print training losses and save logging information to the disk
            losses = model.get_current_losses()
            t_comp = (time.time() - iter_start_time) / opt_pix2pix.batch_size
            visualizer.print_current_losses(epoch, epoch_iter, losses, t_comp, t_data)
            if opt_pix2pix.display_id > 0:
                visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, losses)

        if total_iters % opt_pix2pix.save_latest_freq == 0:   # cache our latest model every <save_latest_freq> iterations
            print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
            save_suffix = 'iter_%d' % total_iters if opt_pix2pix.save_by_iter else 'latest'
            model.save_networks(save_suffix)

        iter_data_time = time.time()
    if epoch % opt_pix2pix.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
        model.save_networks('latest')
        model.save_networks(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' % (epoch, opt_pix2pix.n_epochs + opt_pix2pix.n_epochs_decay, time.time() - epoch_start_time))

learning rate 0.0002000 -> 0.0002000


/home/agysar/miniconda3/envs/advanced_ml/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/home/agysar/miniconda3/envs/advanced_ml/lib/python3.8/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/agysar/miniconda3/envs/advanced_ml/lib/python3.8/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type Interp

(epoch: 1, iters: 100, time: 0.089, data: 0.090) G_GAN: 0.753 G_L1: 66.626 D_real: 0.269 D_fake: 0.329 
(epoch: 1, iters: 200, time: 0.093, data: 0.002) G_GAN: 0.517 G_L1: 36.313 D_real: 0.213 D_fake: 0.164 
(epoch: 1, iters: 300, time: 0.103, data: 0.001) G_GAN: 1.349 G_L1: 47.777 D_real: 0.091 D_fake: 0.110 
(epoch: 1, iters: 400, time: 0.280, data: 0.001) G_GAN: 0.958 G_L1: 47.398 D_real: 0.067 D_fake: 0.113 
End of epoch 1 / 200 	 Time Taken: 24 sec
learning rate 0.0002000 -> 0.0002000


/home/agysar/miniconda3/envs/advanced_ml/lib/python3.8/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


<img src="./checkpoints/pix2pix: LSGAN/newplot.png">

#### Можно заметить, что pix2pix довольно уверенно обучился за 200 эпох. В среднем обучение на одной эпохе заняло 25 секунд.

#### Пример сгенерированных изображений из train_data:
| epoch #  | real image | real stylized image | fake stylized image |
| ------------- | ------------- | ------------- | ------------- |
| __1__  | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch001_real_A.png"> | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch001_real_B.png"> | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch001_fake_B.png"> |
| __50__  | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch050_real_A.png"> | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch050_real_B.png"> | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch050_fake_B.png"> |
| __100__  | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch100_real_A.png"> | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch100_real_B.png"> | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch100_fake_B.png"> |
| __150__  | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch150_real_A.png"> | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch150_real_B.png"> | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch150_fake_B.png"> |
| __200__  | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch200_real_A.png"> | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch200_real_B.png"> | <img src="./checkpoints/pix2pix: LSGAN/web/images/epoch200_fake_B.png"> |

In [3]:
opt_pix2pix_test = TestOptions().parse()  # get test options
opt_pix2pix_test.model = "pix2pix"
opt_pix2pix_test.dataset_mode = "aligned"
opt_pix2pix_test.name = "pix2pix: LSGAN"
opt_pix2pix_test.lambda_L1 = 100.0
# hard-code some parameters for test`
opt_pix2pix_test.num_threads = 0   # test code only supports num_threads = 0
opt_pix2pix_test.batch_size = 1    # test code only supports batch_size = 1
opt_pix2pix_test.serial_batches = True  # disable data shuffling; comment this line if results on randomly chosen images are needed.
opt_pix2pix_test.no_flip = True    # no flip; comment this line if results on flipped images are needed.
opt_pix2pix_test.display_id = -1   # no visdom display; the test code saves the results to a HTML file.
dataset = create_dataset(opt_pix2pix_test)  # create a dataset given opt.dataset_mode and other options
model = create_model(opt_pix2pix_test)      # create a model given opt.model and other options
model.setup(opt_pix2pix_test)               # regular setup: load and print networks; create schedulers

# initialize logger
if opt_pix2pix_test.use_wandb:
    wandb_run = wandb.init(project='CycleGAN-and-pix2pix', name=opt_pix2pix_test.name, config=opt_pix2pix_test) if not wandb.run else wandb.run
    wandb_run._label(repo='CycleGAN-and-pix2pix')

# create a website
web_dir = os.path.join(opt_pix2pix_test.results_dir, opt_pix2pix_test.name, '{}_{}'.format(opt_pix2pix_test.phase, opt_pix2pix_test.epoch))  # define the website directory
if opt_pix2pix_test.load_iter > 0:  # load_iter is 0 by default
    web_dir = '{:s}_iter{:d}'.format(web_dir, opt_pix2pix_test.load_iter)
print('creating web directory', web_dir)
webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (opt_pix2pix_test.name, opt_pix2pix_test.phase, opt_pix2pix_test.epoch))
# test with eval mode. This only affects layers like batchnorm and dropout.
# For [pix2pix]: we use batchnorm and dropout in the original pix2pix. You can experiment it with and without eval() mode.
# For [CycleGAN]: It should not affect CycleGAN as CycleGAN uses instancenorm without dropout.
if opt_pix2pix_test.eval:
    model.eval()
for i, data in enumerate(dataset):
    if i >= opt_pix2pix_test.num_test:  # only apply our model to opt.num_test images.
        break
    model.set_input(data)  # unpack data from data loader
    model.test()           # run inference
    visuals = model.get_current_visuals()  # get image results
    img_path = model.get_image_paths()     # get image paths
    if i % 5 == 0:  # save images to an HTML file
        print('processing (%04d)-th image... %s' % (i, img_path))
    save_images(webpage, visuals, img_path, aspect_ratio=opt_pix2pix_test.aspect_ratio, width=opt_pix2pix_test.display_winsize, use_wandb=opt_pix2pix_test.use_wandb)
webpage.save()  # save the HTML

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./facades/                    
             dataset_mode: single                        
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                             	[default: 0]
 

/home/agysar/miniconda3/envs/advanced_ml/lib/python3.8/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


processing (0000)-th image... ['./facades/test/1.jpg']
processing (0005)-th image... ['./facades/test/103.jpg']
processing (0010)-th image... ['./facades/test/12.jpg']
processing (0015)-th image... ['./facades/test/17.jpg']
processing (0020)-th image... ['./facades/test/21.jpg']
processing (0025)-th image... ['./facades/test/26.jpg']
processing (0030)-th image... ['./facades/test/30.jpg']
processing (0035)-th image... ['./facades/test/35.jpg']
processing (0040)-th image... ['./facades/test/4.jpg']
processing (0045)-th image... ['./facades/test/44.jpg']


#### Но при этом на тесте, результаты удручающие: генерируется шум.

#### Пример сгенерированных изображений из test_data:
|  #  | real image | real stylized image | fake stylized image |
| ------------- | ------------- | ------------- | ------------- |
| __1__  | <img src="./results/pix2pix: LSGAN/test_latest/images/1_real_A.png"> | <img src="./results/pix2pix: LSGAN/test_latest/images/1_real_B.png"> | <img src="./results/pix2pix: LSGAN/test_latest/images/1_fake_B.png"> |
| __2__  | <img src="./results/pix2pix: LSGAN/test_latest/images/2_real_A.png"> | <img src="./results/pix2pix: LSGAN/test_latest/images/2_real_B.png"> | <img src="./results/pix2pix: LSGAN/test_latest/images/2_fake_B.png"> |
| __3__  | <img src="./results/pix2pix: LSGAN/test_latest/images/3_real_A.png"> | <img src="./results/pix2pix: LSGAN/test_latest/images/3_real_B.png"> | <img src="./results/pix2pix: LSGAN/test_latest/images/3_fake_B.png"> |
| __4__  | <img src="./results/pix2pix: LSGAN/test_latest/images/4_real_A.png"> | <img src="./results/pix2pix: LSGAN/test_latest/images/4_real_B.png"> | <img src="./results/pix2pix: LSGAN/test_latest/images/4_fake_B.png"> |
| __10__  | <img src="./results/pix2pix: LSGAN/test_latest/images/10_real_A.png"> | <img src="./results/pix2pix: LSGAN/test_latest/images/10_real_B.png"> | <img src="./results/pix2pix: LSGAN/test_latest/images/10_fake_B.png"> |

## CycleGAN

In [3]:
opt_cyclegan = TrainOptions().parse()

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./facades/                    
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              epoch_count: 1  

In [4]:
opt_cyclegan.model = "cycle_gan"
opt_cyclegan.dataset_mode = "unaligned"
opt_cyclegan.name = "Cyclegan: LSGAN"
opt_cyclegan.lambda_L1 = 100.0
opt_cyclegan.n_epochs = 20

In [5]:
dataset = create_dataset(opt_cyclegan)
dataset_size = len(dataset)    # get the number of images in the dataset.
print('The number of training images = %d' % dataset_size)

dataset [AlignedDataset] was created
The number of training images = 400


In [6]:
model = create_model(opt_cyclegan)      # create a model given opt.model and other options
model.setup(opt_cyclegan)               # regular setup: load and print networks; create schedulers
visualizer = Visualizer(opt_cyclegan)   # create a visualizer that display/save images and plots
total_iters = 0 

Setting up a new session...


initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
model [CycleGANModel] was created
---------- Networks initialized -------------
[Network G_A] Total number of parameters : 11.378 M
[Network G_B] Total number of parameters : 11.378 M
[Network D_A] Total number of parameters : 2.765 M
[Network D_B] Total number of parameters : 2.765 M
-----------------------------------------------
create web directory ./checkpoints/Cyclegan: LSGAN/web...


In [7]:
for epoch in range(opt_cyclegan.epoch_count, opt_cyclegan.n_epochs + opt_cyclegan.n_epochs_decay + 1):    # outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
    epoch_start_time = time.time()  # timer for entire epoch
    iter_data_time = time.time()    # timer for data loading per iteration
    epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch
    visualizer.reset()              # reset the visualizer: make sure it saves the results to HTML at least once every epoch
    model.update_learning_rate()    # update learning rates in the beginning of every epoch.
    for i, data in enumerate(dataset):  # inner loop within one epoch
        iter_start_time = time.time()  # timer for computation per iteration
        if total_iters % opt_cyclegan.print_freq == 0:
            t_data = iter_start_time - iter_data_time

        total_iters += opt_cyclegan.batch_size
        epoch_iter += opt_cyclegan.batch_size
        model.set_input(data)         # unpack data from dataset and apply preprocessing
        model.optimize_parameters()   # calculate loss functions, get gradients, update network weights

        if total_iters % opt_cyclegan.display_freq == 0:   # display images on visdom and save images to a HTML file
            save_result = total_iters % opt_cyclegan.update_html_freq == 0
            model.compute_visuals()
            visualizer.display_current_results(model.get_current_visuals(), epoch, save_result)

        if total_iters % opt_cyclegan.print_freq == 0:    # print training losses and save logging information to the disk
            losses = model.get_current_losses()
            t_comp = (time.time() - iter_start_time) / opt_cyclegan.batch_size
            visualizer.print_current_losses(epoch, epoch_iter, losses, t_comp, t_data)
            if opt_cyclegan.display_id > 0:
                visualizer.plot_current_losses(epoch, float(epoch_iter) / dataset_size, losses)

        if total_iters % opt_cyclegan.save_latest_freq == 0:   # cache our latest model every <save_latest_freq> iterations
            print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
            save_suffix = 'iter_%d' % total_iters if opt_cyclegan.save_by_iter else 'latest'
            model.save_networks(save_suffix)

        iter_data_time = time.time()
    if epoch % opt_cyclegan.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
        model.save_networks('latest')
        model.save_networks(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' % (epoch, opt_cyclegan.n_epochs + opt_cyclegan.n_epochs_decay, time.time() - epoch_start_time))

learning rate 0.0002000 -> 0.0002000


/home/agysar/miniconda3/envs/advanced_ml/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/home/agysar/miniconda3/envs/advanced_ml/lib/python3.8/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


<img src="./checkpoints/Cyclegan: LSGAN/newplot (1).png">

#### В свою очередь CycleGAN обучается почти в 5 раз медленнее чем pix2pix. В среднем обучение на одной эпохе заняло 120 секунд. 
#### При этом качество генерируемых изображений сопоставимо с pix2pix уже на 25 эпохе.

#### Пример сгенерированных изображений:
| epoch #  | real image | real stylized image | fake stylized image |
| ------------- | ------------- | ------------- | ------------- |
| __1__  | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch001_real_A.png"> | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch001_real_B.png"> | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch001_rec_B.png"> |
| __5__  | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch005_real_A.png"> | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch005_real_B.png"> | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch005_rec_B.png"> |
| __10__  | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch010_real_A.png"> | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch010_real_B.png"> | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch010_rec_B.png"> |
| __15__  | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch015_real_A.png"> | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch015_real_B.png"> | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch015_rec_B.png"> |
| __25__  | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch025_real_A.png"> | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch025_real_B.png"> | <img src="./checkpoints/Cyclegan: LSGAN/web/images/epoch025_rec_B.png"> |

In [4]:
opt_cyclegan_test = TestOptions().parse()  # get test options
opt_cyclegan_test.model = "cycle_gan"
opt_cyclegan_test.dataset_mode = "aligned"
opt_cyclegan_test.name = "Cyclegan: LSGAN"
opt_cyclegan_test.lambda_L1 = 100.0
# hard-code some parameters for test`
opt_cyclegan_test.num_threads = 0   # test code only supports num_threads = 0
opt_cyclegan_test.batch_size = 1    # test code only supports batch_size = 1
opt_cyclegan_test.serial_batches = True  # disable data shuffling; comment this line if results on randomly chosen images are needed.
opt_cyclegan_test.no_flip = True    # no flip; comment this line if results on flipped images are needed.
opt_cyclegan_test.display_id = -1   # no visdom display; the test code saves the results to a HTML file.
dataset = create_dataset(opt_cyclegan_test)  # create a dataset given opt.dataset_mode and other options
model = create_model(opt_cyclegan_test)      # create a model given opt.model and other options
model.setup(opt_cyclegan_test)               # regular setup: load and print networks; create schedulers

# initialize logger
if opt_cyclegan_test.use_wandb:
    wandb_run = wandb.init(project='CycleGAN-and-pix2pix', name=opt_cyclegan_test.name, config=opt_cyclegan_test) if not wandb.run else wandb.run
    wandb_run._label(repo='CycleGAN-and-pix2pix')

# create a website
web_dir = os.path.join(opt_cyclegan_test.results_dir, opt_cyclegan_test.name, '{}_{}'.format(opt_cyclegan_test.phase, opt_cyclegan_test.epoch))  # define the website directory
if opt_cyclegan_test.load_iter > 0:  # load_iter is 0 by default
    web_dir = '{:s}_iter{:d}'.format(web_dir, opt_cyclegan_test.load_iter)
print('creating web directory', web_dir)
webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (opt_cyclegan_test.name, opt_cyclegan_test.phase, opt_cyclegan_test.epoch))
# test with eval mode. This only affects layers like batchnorm and dropout.
# For [cyclegan]: we use batchnorm and dropout in the original cyclegan. You can experiment it with and without eval() mode.
# For [CycleGAN]: It should not affect CycleGAN as CycleGAN uses instancenorm without dropout.
if opt_cyclegan_test.eval:
    model.eval()
for i, data in enumerate(dataset):
    if i >= opt_cyclegan_test.num_test:  # only apply our model to opt.num_test images.
        break
    model.set_input(data)  # unpack data from data loader
    model.test()           # run inference
    visuals = model.get_current_visuals()  # get image results
    img_path = model.get_image_paths()     # get image paths
    if i % 5 == 0:  # save images to an HTML file
        print('processing (%04d)-th image... %s' % (i, img_path))
    save_images(webpage, visuals, img_path, aspect_ratio=opt_cyclegan_test.aspect_ratio, width=opt_cyclegan_test.display_winsize, use_wandb=opt_cyclegan_test.use_wandb)
webpage.save()  # save the HTML

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./facades/                    
             dataset_mode: single                        
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                             	[default: 0]
 

/home/agysar/miniconda3/envs/advanced_ml/lib/python3.8/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


processing (0000)-th image... ['./facades/test/1.jpg']
processing (0005)-th image... ['./facades/test/103.jpg']
processing (0010)-th image... ['./facades/test/12.jpg']
processing (0015)-th image... ['./facades/test/17.jpg']
processing (0020)-th image... ['./facades/test/21.jpg']
processing (0025)-th image... ['./facades/test/26.jpg']
processing (0030)-th image... ['./facades/test/30.jpg']
processing (0035)-th image... ['./facades/test/35.jpg']
processing (0040)-th image... ['./facades/test/4.jpg']
processing (0045)-th image... ['./facades/test/44.jpg']


#### В отличие от pix2pix, CycleGAN может улавливать контуры стиля уже после 25+ эпох.

#### Пример сгенерированных изображений из test_data:
|  #  | real image | real stylized image | fake stylized image |
| ------------- | ------------- | ------------- | ------------- |
| __1__  | <img src="./results/Cyclegan: LSGAN/test_latest/images/1_real_A.png"> | <img src="./results/Cyclegan: LSGAN/test_latest/images/1_real_B.png"> | <img src="./results/Cyclegan: LSGAN/test_latest/images/1_fake_B.png"> |
| __2__  | <img src="./results/Cyclegan: LSGAN/test_latest/images/2_real_A.png"> | <img src="./results/Cyclegan: LSGAN/test_latest/images/2_real_B.png"> | <img src="./results/Cyclegan: LSGAN/test_latest/images/2_fake_B.png"> |
| __3__  | <img src="./results/Cyclegan: LSGAN/test_latest/images/3_real_A.png"> | <img src="./results/Cyclegan: LSGAN/test_latest/images/3_real_B.png"> | <img src="./results/Cyclegan: LSGAN/test_latest/images/3_fake_B.png"> |
| __4__  | <img src="./results/Cyclegan: LSGAN/test_latest/images/4_real_A.png"> | <img src="./results/Cyclegan: LSGAN/test_latest/images/4_real_B.png"> | <img src="./results/Cyclegan: LSGAN/test_latest/images/4_fake_B.png"> |
| __10__  | <img src="./results/Cyclegan: LSGAN/test_latest/images/10_real_A.png"> | <img src="./results/Cyclegan: LSGAN/test_latest/images/10_real_B.png"> | <img src="./results/Cyclegan: LSGAN/test_latest/images/10_fake_B.png"> |